## Exponential Smoothing(指数平滑模型)

#### Simple Exponential Smoothing(SES, 简单指数平滑)
- 适用于预测没有明显趋势或季节因素的数据
- 预测使用加权平均来计算，最大的权重与最近的观测值相关，而最小的权重与最远的观测值相关
- $$\hat{y}_{T+1｜T}=\alpha y_T+\alpha (1-\alpha) y_{T-1}+\alpha (1-\alpha)^2 y_{T-2}+\dots$$
- 权重减小率由平滑参数$\alpha (0\leq \alpha\leq 1)$控制。如果$\alpha$很大(即接近1)，则表示对更近期的观察给予更多权重。
- 若$\alpha =0$，则预测值应取历史数据的平均值，即平均值法
- 若$\alpha =1$, 则简单地将预测值设置为最后一次观测的值，即朴素方法

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams['font.family'] = 'Heiti TC'
plt.rcParams['axes.unicode_minus']=False

In [ ]:
data = pd.read_excel("sale_data.xlsx")
data.set_index('时间', inplace=True)
data.head()

In [ ]:
plt.figure(figsize=(20,6))

# 在fit1中，平滑参数设置为 0.2
fit1 = SimpleExpSmoothing(data).fit(smoothing_level=0.2,optimized=False)
l1, = plt.plot(list(fit1.fittedvalues) + list(fit1.forecast(5)), marker='o')
# 在fit2中，平滑参数设置为 0.6
fit2 = SimpleExpSmoothing(data).fit(smoothing_level=0.6,optimized=False)
l2, = plt.plot(list(fit2.fittedvalues) + list(fit2.forecast(5)), marker='o')
# (推荐)在fit3中，不设置平滑参数，使用自动优化，允许statsmodels自动为我们找到优化值。
fit3 = SimpleExpSmoothing(data).fit()
l3, = plt.plot(list(fit3.fittedvalues) + list(fit3.forecast(5)), marker='o')
# 原始数据
l4, = plt.plot(data, marker='o')

plt.legend(handles=[l1, l2, l3, l4], labels=['a=0.2', 'a=0.6', 'auto', 'data'], loc='best', prop={'size':16})
plt.xticks(rotation=17, fontsize=12)
plt.show()

#### Holt's 方法(二次指数平滑)
- Holt扩展了简单的指数平滑(数据解决方案没有明确的趋势或季节性)，以便在1957年预测数据趋势
- Holt的方法包括预测方程和两个平滑方程(一个用于水平，一个用于趋势):
    - $\text{Forecast Equation}$ $$\hat{y}_{t+h|t} = l_t+hb_t$$
    - $\text{Level Equation}$ $$l_t=\alpha y_t + (1-\alpha)(l_{t-1}+b_{t-1})$$
    - $\text{Trend Equation}$ $$b_t=\beta^*(l_t-l_{t-1})+(1-\beta^*)b_{t-1}$$
    - 其中 $0\leq \alpha \leq 1$ 是水平平滑参数，$0\leq \beta^* \leq 1$是趋势平滑参数
- 对于长期预测，使用Holt方法的预测在未来会无限期地增加或减少。在这种情况下，我们使用具有阻尼参数0<φ<1的阻尼趋势方法来防止预测“失控”。
    - $\text{Forecast Equation}$ $$\hat{y}_{t+h|t} = l_t+(\phi + \phi^2 + \dots + \phi^h)b_t$$
    - $\text{Level Equation}$ $$l_t=\alpha y_t + (1-\alpha)(l_{t-1}+ \phi b_{t-1})$$
    - $\text{Trend Equation}$ $$b_t=\beta^*(l_t-l_{t-1})+(1-\beta^*) \phi b_{t-1}$$
    - 其中 $0\leq \alpha \leq 1$ 是水平平滑参数，$0\leq \beta^* \leq 1$是趋势平滑参数

In [ ]:
from statsmodels.tsa.holtwinters import Holt
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams['font.family'] = 'Heiti TC'
plt.rcParams['axes.unicode_minus']=False

In [ ]:
data = pd.read_excel("sale_data.xlsx")
data.set_index('时间', inplace=True)
data.head()

In [ ]:
plt.figure(figsize=(20,6))

# 在fit1中，平滑参数α=0.8，β∗=0.2
fit1 = Holt(data).fit(smoothing_level=0.8, smoothing_slope=0.2, optimized=False)
l1, = plt.plot(list(fit1.fittedvalues) + list(fit1.forecast(5)), marker='^')
# 在fit2中，使用指数模型而不是Holt的加法模型（默认）
fit2 = Holt(data, exponential=True).fit(smoothing_level=0.8, smoothing_slope=0.2, optimized=False)
l2, = plt.plot(list(fit2.fittedvalues) + list(fit2.forecast(5)), marker='.')
# 在fit3中，使用阻尼版本的Holt附加模型，但允许优化阻尼参数φ，同时固定α=0.8，β∗=0.2的值
fit3 = Holt(data, damped=True).fit(smoothing_level=0.8, smoothing_slope=0.2)
l3, = plt.plot(list(fit3.fittedvalues) + list(fit3.forecast(5)), marker='.')
# 原始数据
l4, = plt.plot(data, marker='.')

plt.legend(handles = [l1, l2, l3, l4], labels = ["Holt's linear trend", "Exponential trend", "Additive damped trend", 'data'], loc = 'best', prop={'size': 16})
plt.xticks(rotation=17, fontsize=12)
plt.show()

#### Holt-Winters 方法(三次指数平滑)
- Holt-Winters的方法适用于具有趋势和季节性的数据，其包括季节性平滑参数γ。 
- 此方法有两种变体：
    - 加法方法：整个序列的季节变化基本保持不变
    - 乘法方法：季节变化与系列水平成比例变化

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams['font.family'] = 'Heiti TC'
plt.rcParams['axes.unicode_minus']=False

In [ ]:
data = pd.read_excel("sale_data.xlsx")
data.set_index('时间', inplace=True)
data.head()

In [ ]:
plt.figure(figsize=(20,6))
data_sr = data

# 在fit1中，我们使用加法趋势，周期 season_length=4 的加性季节和Box-Cox变换。
fit1 = ExponentialSmoothing(data_sr, seasonal_periods=4, trend='add', seasonal='add', use_boxcox=True).fit()
l1, = plt.plot(list(fit1.fittedvalues) + list(fit1.forecast(5)), marker='^')

# 在fit2中，我们使用加法趋势，周期 season_length=4 的乘法季节和Box-Cox变换。
fit2 = ExponentialSmoothing(data_sr, seasonal_periods=4, trend='add', seasonal='mul', use_boxcox=True).fit()
l2, = plt.plot(list(fit2.fittedvalues) + list(fit2.forecast(5)), marker='*')

# 在fit3中，我们使用加性阻尼趋势，周期 season_length=4 的加性季节和Box-Cox变换。
fit3 = ExponentialSmoothing(data_sr, seasonal_periods=4, trend='add', seasonal='add', damped=True, use_boxcox=True).fit()
l3, = plt.plot(list(fit3.fittedvalues) + list(fit3.forecast(5)), marker='.')

# 在fit4中，我们使用加性阻尼趋势，周期 season_length=4 的乘法季节和Box-Cox变换。
fit4 = ExponentialSmoothing(data_sr, seasonal_periods=4, trend='add', seasonal='mul', damped=True, use_boxcox=True).fit()
l4, = plt.plot(list(fit4.fittedvalues) + list(fit4.forecast(5)), marker='.')

# 原始数据
l5, = plt.plot(data, marker='.')

plt.legend(handles=[l1, l2, l3, l4, l5], labels=["aa", "am", "aa damped", "am damped","data"], loc='best', prop={'size': 16})
plt.xticks(rotation=17, fontsize=12)
plt.show()

#### Autoregressive Moving Average Model(ARMA, 自回归移动平均模型)
- ARMA模型将自回归(AR)和移动平均(MA)两种方法结合起来，用于描述时间序列数据的动态特性。
- 对于平稳的非白噪声序列，它的均值和方差是常数，ARMA是最常用的平稳序列拟合模型。
- 而对于非平稳序列，均值方差不稳定，一般将其转为平稳序列。
- ARMA模型的一般形式可以表示为ARMA(p, q)，其中p代表自回归阶数，q代表移动平均阶数。
    - AR部分表示时间序列的当前值与过去p个值的线性组合，可以表示为：
    $$X_t=c+{\phi}_1 X_{t-1}+{\phi}_2 X_{t-2}+ \dots +{\phi}_p X_{t-p}+{\varepsilon}_t $$
    - MA部分表示时间序列的当前值与过去q个滞后误差的线性组合，可以表示为：
    $$X_t = \mu+ {\varepsilon}_t + {\theta}_1 {\varepsilon}_{t-1}+{\theta}_2 {\varepsilon}_{t-2} + \dots + {\theta}_q {\varepsilon}_{t-q}$$
    - 其中，$X_t$表示时间序列的当前值，${\varepsilon}_t$表示当前时刻的误差，${\phi}_i$和${\theta}_i$表示自回归系数和移动平均系数，$c$和$\mu$表示常数项。
    - 则ARMA模型可表示为：
    $$X_t={\varepsilon}_t+{\phi}_1 X_{t-1}+{\phi}_2 X_{t-2}+\dots+{\phi}_p X_{t-p}+{\theta}_1{\varepsilon}_{t-1}+{\theta}_2{\varepsilon}_{t-2}+\dots+{\theta}_q{\varepsilon}_{t-q}$$

#### Autoregressive Integrated Moving Average Model(ARIMA, 差分自回归移动平均模型)
- 如果一个序列差分后具有平稳性，则该序列为差分平稳序列，可用ARIMA

In [ ]:
from statsmodels.graphics.tsaplots import *
from statsmodels.tsa.stattools import adfuller as ADF
from matplotlib import pylab
import matplotlib.dates as mdate

pylab.rcParams['figure.figsize'] = (10, 6)

In [ ]:
dateparse = lambda dates:pd.datetime.strptime(dates, '%m/%d/%Y')
data = pd.read_csv('stock.csv', parse_dates=['时间'], date_parser=dateparse, index_col=0)
data.head()

In [ ]:
fig, ax = plt.subplots()
ax.xaxis.set_major_formatter(mdate.DateFormatter('%Y-%m-%d'))
plt.xticks(pd.date_range(data.index[0],data.index[-1],freq='3M'),rotation=20, fontsize=12)
plt.plot(data)
plt.show()

In [ ]:
# 自相关图
plot_acf(data).show()

In [ ]:
# 偏自相关图
plot_pacf(data).show()

In [ ]:
D_data = data.diff().dropna()   #1阶差分，丢弃na值
D_data.columns = [u'一阶差分']  #更改列名
D_data.plot() #时序图
plt.show()

In [ ]:
plot_acf(D_data, lags=4).show()

In [ ]:
#平稳性检测
print(u'原始序列的ADF检验结果为：', ADF(data))

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1=fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(data,lags=40,ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(data,lags=40,ax=ax2)